In [22]:
import numpy as np
import time 
import pandas as pd

In [23]:
# all the merging and filtering should be done before passing the function

In [24]:
# required colums

#selected=all_rig_data[['Item','Site','Maximum Level','Minimum Level','Average Cost','Current Balance','Commodity Group','Group_ID']]


In [25]:
def adjustMinMax(dataFrame):
    # converting the data frame 
    data=pd.DataFrame(dataFrame)
    # data frame with non zero minimum
    df_non_zero=pd.DataFrame(data[data['Minimum Level']!=0])
    # data frame with zero min
    df_zero_min=pd.DataFrame(data[data['Minimum Level']==0])
    
    # replacing the new min as old min for rows with zero min 
    df_zero_min['Maximum Level Adjusted']=df_zero_min['Maximum Level']
    df_zero_min['Minimum Level Adjusted']=df_zero_min['Minimum Level']
   
    # counting unique items
    
    vc = df_non_zero.Item.value_counts()
    
    repeated=vc[vc>1].index.tolist()
    non_repeated=vc[vc==1].index.tolist()
    
    # data frame where the items are repeated
    repeated_data=df_non_zero[df_non_zero['Item'].isin(repeated)]
    
    # data frame where the items are non repeated
    non_repeated_data=pd.DataFrame(df_non_zero[df_non_zero['Item'].isin(non_repeated)])
    
    non_repeated_data['Maximum Level Adjusted']=non_repeated_data['Maximum Level']
    non_repeated_data['Minimum Level Adjusted']=non_repeated_data['Minimum Level']
    
    t = time.time()
    # do stuff
    new_max_min_snippets=[]


    for name, group in repeated_data.groupby('Item'):
        max_min_snippet = group
        #print(max_min_snippet)
        min_level=max_min_snippet.min(0)
        min_all=[max_min_snippet[col][max_min_snippet[col].ne(0)].min() for col in max_min_snippet.columns]
        min_level=min_all[3]
        #print(\" the min value is *******\",min_level)
        new_max_min_snippet=max_min_snippet.copy()
        #print(max_min_snippet)
        new_max_min_snippet['Minimum Level Adjusted']=max_min_snippet['Minimum Level']
        new_max_min_snippet['Current Balance']=max_min_snippet['Current Balance']
        new_max_min_snippet.loc[new_max_min_snippet['Minimum Level'] !=0 , 'Minimum Level Adjusted'] = min_level
        min_difference=new_max_min_snippet['Minimum Level']-new_max_min_snippet['Minimum Level Adjusted']
        new_max_min_snippet['Maximum Level Adjusted']= np.array(new_max_min_snippet['Maximum Level'])-np.array(min_difference)
            #print(new_max_min_snippet)
        new_max_min_snippets.append(new_max_min_snippet)
            #print(len(new_max_min_snippets))
        Data_frame = pd.concat(new_max_min_snippets, ignore_index=True)
    
    elapsed = time.time() - t
    print(elapsed)
    
    # concating the data frames
    frames=[Data_frame,non_repeated_data,df_zero_min]
    
    FinalDF = pd.concat(frames,sort=False)
    
    return(FinalDF)
    
    
    
    
    

In [26]:
all_rig_data=pd.read_csv("All_rigs_with_name.csv", low_memory=False)

In [27]:
    
all_rig_data['Current Balance'] = all_rig_data['Current Balance'].replace(to_replace =",", value = '', regex = True) 

all_rig_data['Maximum Level'] = all_rig_data['Maximum Level'].replace(to_replace =",", value = '', regex = True) 

all_rig_data['Minimum Level'] = all_rig_data['Minimum Level'].replace(to_replace =",", value = '', regex = True) 

all_rig_data['Average Cost'] = all_rig_data['Average Cost'].replace(to_replace =",", value = '', regex = True) 

    # converting to float for calculations
    
all_rig_data['Average Cost'] = all_rig_data['Average Cost'].astype(float)

all_rig_data['Minimum Level'] = all_rig_data['Minimum Level'].astype(float)

all_rig_data['Maximum Level'] = all_rig_data['Maximum Level'].astype(float)

all_rig_data['Current Balance'] = all_rig_data['Current Balance'].astype(float)

In [28]:
all_rig_data.shape

(150742, 28)

In [29]:
selected=all_rig_data[['Item','Site','Maximum Level','Minimum Level','Average Cost','Current Balance','Commodity Group','Group_ID']]

In [30]:
Group_4=selected.loc[selected['Group_ID'] == 'Group_4']

In [31]:
Group_4.shape

(37934, 8)

In [32]:
X=adjustMinMax(Group_4)

520.6260931491852


In [33]:
#Similarly we can do for all the groups of rigs

In [34]:
X.head()

,Item,Site,Maximum Level,Minimum Level,Average Cost,Current Balance,Commodity Group,Group_ID,Minimum Level Adjusted,Maximum Level Adjusted
0,1016771,2298,8.0,6.0,13.30,4.0,ESP,Group_4,1.0,3.0
1,1016771,2299,2.0,1.0,26.38,2.0,ESP,Group_4,1.0,2.0
2,1018639,2301,8.0,4.0,1.04,9.0,ESP,Group_4,3.0,7.0
3,1018639,6070,6.0,3.0,1.05,6.0,ESP,Group_4,3.0,6.0
4,1096934,2298,4.0,2.0,1439.21,3.0,ESP,Group_4,2.0,4.0
